In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [39]:
patients_bpn = pd.read_csv("pn_bact/patients_id.csv", index_col=0)
patients_vpn = pd.read_csv("pn_virus/patients_id.csv", index_col=0)

In [32]:
print(patients_vpn.shape[0]+patients_bpn.shape[0],
      patients_vpn.shape[0],patients_vpn.shape[0]/(patients_vpn.shape[0]+patients_bpn.shape[0]),
      patients_bpn.shape[0],patients_bpn.shape[0]/(patients_vpn.shape[0]+patients_bpn.shape[0]))

2502 463 0.1850519584332534 2039 0.8149480415667466


# 获取基础信息

## 感染类型分析

In [40]:
code_diag = pd.read_csv("mimic-iv-2/hosp/d_icd_diagnoses.csv")

In [27]:
diag_bpn = pd.read_csv("pn_bact/sofa_diagnoses.csv", index_col=0)
code_bpn = pd.merge(diag_bpn, code_diag)
code_bpn["long_title"] = code_bpn["long_title"].str.lower()
code_bpn.loc[code_bpn["long_title"]=="bacterial pneumonia, unspecified", 
             "long_title"] = "pneumonia due to unspecified bacteria"
code_bpn.loc[code_bpn["long_title"]=="unspecified bacterial pneumonia", 
             "long_title"] = "pneumonia due to unspecified bacteria"
code_bpn.loc[code_bpn["long_title"]=="pneumonia due to other specified organism", 
             "long_title"] = "pneumonia due to other specified bacteria"
code_bpn.loc[code_bpn["long_title"]=="methicillin resistant pneumonia due to staphylococcus aureus", 
             "long_title"] = "pneumonia due to methicillin resistant staphylococcus aureus"
code_bpn.loc[code_bpn["long_title"]=="methicillin susceptible pneumonia due to staphylococcus aureus", 
             "long_title"] = "pneumonia due to methicillin susceptible staphylococcus aureus"
code_bpn.loc[code_bpn["long_title"]=="other staphylococcus pneumonia", 
             "long_title"] = "pneumonia due to other staphylococcus"
code_bpn.loc[code_bpn["long_title"]=="pneumonia due to staphylococcus, unspecified", 
             "long_title"] = "pneumonia due to other staphylococcus"
code_bpn.loc[code_bpn["long_title"]=="pneumonia due to escherichia coli [e. coli]", 
             "long_title"] = "pneumonia due to escherichia coli"
code_bpn.loc[code_bpn["long_title"]=="pneumonia due to other streptococci", 
             "long_title"] = "pneumonia due to other streptococcus"
code_bpn.loc[code_bpn["long_title"]=="pneumonia due to streptococcus, unspecified", 
             "long_title"] = "pneumonia due to other streptococcus"
code_bpn.loc[code_bpn["long_title"]=="pneumonia due to streptococcus, group a", 
             "long_title"] = "pneumonia due to other streptococcus"
code_bpn.loc[code_bpn["long_title"]=="pneumonia due to streptococcus, group b", 
             "long_title"] = "pneumonia due to other streptococcus"
code_bpn.loc[code_bpn["long_title"]=="pneumonia due to hemophilus influenzae [h. influenzae]", 
             "long_title"] = "pneumonia due to hemophilus influenzae"
code_bpn = pd.merge(patients_bpn,code_bpn[['subject_id', 'hadm_id','long_title']])
code_bpn.to_csv("pn_bact/code_bpn.csv", header=True,index=True)
counts = pd.DataFrame(code_bpn["long_title"].value_counts())
counts["per"] = counts["long_title"]/len(set(code_bpn["hadm_id"]))
counts.columns = ["次数","频率"]
round(counts,2)

In [41]:
diag_vpn = pd.read_csv("pn_virus/sofa_diagnoses.csv", index_col=0)
code_vpn = pd.merge(diag_vpn, code_diag)
code_vpn["long_title"] = code_vpn["long_title"].str.lower()
code_vpn.loc[code_vpn["long_title"].str.contains("other identified influenza"), 
             "long_title"] = "influenza due to other influenza virus"
code_vpn.loc[code_vpn["long_title"].str.contains("unidentified influenza virus"), 
             "long_title"] = "influenza due to other influenza virus"
code_vpn.loc[code_vpn["long_title"].str.contains("influenza due to identified avian influenza virus"),
             "long_title"] = "influenza due to identified avian influenza virus"
code_vpn.loc[code_vpn["long_title"].str.contains("influenza due to identified novel influenza"), 
             "long_title"] = "influenza due to identified novel influenza"
code_vpn.loc[code_vpn["long_title"].str.contains("influenza with"), 
             "long_title"] = "influenza due to influenza"
code_vpn.loc[code_vpn["long_title"].str.contains("influenza due to other influenza virus"), 
             "long_title"] = "influenza due to influenza"
code_vpn.loc[code_vpn["long_title"].str.contains("influenza due to identified novel influenza"), 
             "long_title"] = "influenza due to influenza"
code_vpn.loc[code_vpn["long_title"].str.contains("h1n1"), 
             "long_title"] = "influenza due to identified h1n1 influenza virus"
code_vpn.loc[code_vpn["long_title"]=="parainfluenza", 
             "long_title"] = "pneumonia due to parainfluenza virus"
code_vpn.loc[code_vpn["long_title"]=="human metapneumovirus pneumonia", 
             "long_title"] = "pneumonia due to human metapneumovirus pneumonia"
code_vpn.loc[code_vpn["long_title"]=="adenoviral pneumonia", 
             "long_title"] = "pneumonia due to adenovirus"
code_vpn.loc[code_vpn["long_title"]=="parainfluenza virus pneumonia", 
             "long_title"] = "pneumonia due to parainfluenza virus"
code_vpn.loc[code_vpn["long_title"]=="respiratory syncytial virus pneumonia", 
             "long_title"] = "pneumonia due to respiratory syncytial virus"
code_vpn.loc[code_vpn["long_title"].isin(["other viral pneumonia",'viral pneumonia, unspecified',"pneumonia due to other virus not elsewhere classified"]), 
             "long_title"] = "pneumonia due to other virus"
code_vpn = pd.merge(patients_vpn,code_vpn[['subject_id', 'hadm_id','long_title']])
code_vpn.to_csv("pn_virus/code_vpn.csv", header=True,index=True)
counts = pd.DataFrame(code_vpn["long_title"].value_counts())
counts["per"] = counts["long_title"]/len(set(code_bpn["hadm_id"]))
round(counts,2)

# 基础信息

In [190]:
full_vpn = patients_vpn
full_bpn = patients_bpn
dic = {}

In [191]:
# 年龄信息
var = pd.read_csv("sofa_data/age.csv", header=0)
i = "age"
var_vpn = pd.merge(patients_vpn,var)
var_bpn = pd.merge(patients_bpn,var)
print(var_vpn.shape, var_bpn.shape)

var_pn = pd.concat((var_vpn, var_bpn),0)

dic[i] = ["%.1f(%.1f,%.1f)" % (
                np.median(var_pn[i]), 
                np.percentile(var_pn[i], (25), interpolation='midpoint'),
                np.percentile(var_pn[i], (75), interpolation='midpoint')),
          "%.1f(%.1f,%.1f)" % (
                np.median(var_vpn[i]), 
                np.percentile(var_vpn[i], (25), interpolation='midpoint'),
                np.percentile(var_vpn[i], (75), interpolation='midpoint')),
          "%.1f(%.1f,%.1f)" % (
                np.median(var_bpn[i]), 
                np.percentile(var_bpn[i], (25), interpolation='midpoint'),
                np.percentile(var_bpn[i], (75), interpolation='midpoint')),
          "%.2f" % (stats.mannwhitneyu(var_vpn[i].T,var_bpn[i].T,alternative='two-sided').pvalue)]
full_vpn = pd.merge(full_vpn, var)
full_bpn = pd.merge(full_bpn, var)

(463, 7) (2039, 7)


/tmp/ipykernel_1980093/1056497005.py:8: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  var_pn = pd.concat((var_vpn, var_bpn),0)


In [192]:
# 性别信息
var = pd.read_csv("mimic-iv-2/hosp/patients.csv")
i = "gender"
var.loc[var[i]=="F",i] = 1
var.loc[var[i]=="M",i] = 0

var_vpn = pd.merge(patients_vpn,var)
var_bpn = pd.merge(patients_bpn,var)
print(var_vpn.shape, var_bpn.shape)

var_pn = pd.concat((var_vpn, var_bpn),0)
dic[i] = [
        "%s(%.1f)" % (np.sum(var_pn[i]), 
                        np.mean(var_pn[i])*100),
        "%s(%.1f)" % (np.sum(var_vpn[i]), 
                        np.mean(var_vpn[i])*100),
        "%s(%.1f)" % (np.sum(var_bpn[i]), 
                        np.mean(var_bpn[i])*100),
        "%.2f" % (stats.chi2_contingency([[np.sum(var_vpn[i]), patients_vpn.shape[0]-np.sum(var_vpn[i])],
                [np.sum(var_bpn[i]), patients_bpn.shape[0]-np.sum(var_bpn[i])]],correction=False)[1])]

full_vpn = pd.merge(full_vpn, var)
full_bpn = pd.merge(full_bpn, var)

(463, 8) (2039, 8)


/tmp/ipykernel_1980093/1014212670.py:11: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  var_pn = pd.concat((var_vpn, var_bpn),0)


In [193]:
# 体重
var = pd.read_csv("sofa_data/first_day_weight.csv")
i = "weight"
var_vpn = pd.merge(patients_vpn,var)
var_bpn = pd.merge(patients_bpn,var)
print(var_vpn.shape, var_bpn.shape)
var_pn = pd.concat((var_vpn, var_bpn),0)

dic[i] = ["%.1f(%.1f,%.1f)" % (
                np.median(var_pn[i]), 
                np.percentile(var_pn[i], (25), interpolation='midpoint'),
                np.percentile(var_pn[i], (75), interpolation='midpoint')),
          "%.1f(%.1f,%.1f)" % (
                np.median(var_vpn[i]), 
                np.percentile(var_vpn[i], (25), interpolation='midpoint'),
                np.percentile(var_vpn[i], (75), interpolation='midpoint')),
          "%.1f(%.1f,%.1f)" % (
                np.median(var_bpn[i]), 
                np.percentile(var_bpn[i], (25), interpolation='midpoint'),
                np.percentile(var_bpn[i], (75), interpolation='midpoint')),
          "%.2f" % (stats.mannwhitneyu(var_vpn[i].T,var_bpn[i].T,alternative='two-sided').pvalue)]

full_vpn = pd.merge(full_vpn, var)
full_bpn = pd.merge(full_bpn, var)

(463, 7) (2039, 7)


/tmp/ipykernel_1980093/3079077602.py:7: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  var_pn = pd.concat((var_vpn, var_bpn),0)


In [219]:
# 机械通气
var = pd.read_csv("sofa_data/ventilation.csv")
i = "ventilation_status"
var.loc[var[i].isin(['HFNC','None','SupplementalOxygen']),i] = 0
var.loc[var[i].isin(['InvasiveVent','NonInvasiveVent','Tracheostomy']),i] = 1
var = var[['stay_id','ventilation_status']]
var_vpn = pd.merge(patients_vpn,var)
var_vpn = var_vpn[['stay_id',"ventilation_status"]].groupby(['stay_id']).agg(sum).reset_index()
var_vpn.loc[var_vpn["ventilation_status"]>0,"ventilation_status"] = 1

var_bpn = pd.merge(patients_bpn,var)
var_bpn = var_bpn[['stay_id',"ventilation_status"]].groupby(['stay_id']).agg(sum).reset_index()
var_bpn.loc[var_bpn["ventilation_status"]>0,"ventilation_status"] = 1

var_pn = pd.concat((var_vpn, var_bpn),0)

dic[i] = [
        "%s(%.1f)" % (np.sum(var_pn)[0],np.sum(var_pn)[0]/(patients_vpn.shape[0]+patients_bpn.shape[0]) *100),
        "%s(%.1f)" % (np.sum(var_vpn)[0],np.sum(var_vpn)[0]/patients_vpn.shape[0]*100),
        "%s(%.1f)" % (np.sum(var_bpn)[0],np.sum(var_bpn)[0]/patients_bpn.shape[0]*100),
        "%.2f" % (stats.chi2_contingency([[np.sum(var_vpn)[0], patients_vpn.shape[0]-np.sum(var_vpn)[0]],
                [np.sum(var_bpn)[0], patients_bpn.shape[0]-np.sum(var_bpn)[0]]],correction=False)[1])]
full_vpn = pd.merge(full_vpn, var_vpn,"outer")
full_bpn = pd.merge(full_bpn, var_bpn, "outer")

/tmp/ipykernel_1980093/335916634.py:20: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  var_pn = pd.concat((var_vpn, var_bpn),0)


## 合并症状

In [ ]:
diagnoses = pd.read_csv("mimic-iv-2/hosp/diagnoses_icd.csv", header=0)
comd = {}
# Congestive heart failure
comd["Congestive heart failure"] = pd.concat((diagnoses.loc[diagnoses["icd_code"].str.match("428\d"), :],
                          diagnoses.loc[diagnoses["icd_code"].str.match("I50\d"), :]),0)
# Cardiacarrhythmias
comd["Cardiacarrhythmias"] = pd.concat((diagnoses.loc[diagnoses["icd_code"].str.match("427\d"), :],
                          diagnoses.loc[diagnoses["icd_code"].str.match("I5[4-9]\d"), :]),0)
# Pulmonarycirculation
comd["Pulmonarycirculation"] = pd.concat((diagnoses.loc[diagnoses["icd_code"].str.match("41[5-7]\d"), :],
                          diagnoses.loc[diagnoses["icd_code"].str.match("I2[6-8]\d"), :]),0)
# Hypertention
comd["Hypertention"] = pd.concat((diagnoses.loc[diagnoses["icd_code"].str.match("40\d"), :],
                          diagnoses.loc[diagnoses["icd_code"].str.match("I1\d"), :]),0)
# Chronicpulmonary
comd["Chronicpulmonary"] = pd.concat((diagnoses.loc[diagnoses["icd_code"].str.match("49\d"), :],
                          diagnoses.loc[diagnoses["icd_code"].str.match("J4[0-7]\d"), :]),0)
# Renal failure
comd["Renal failure"] = pd.concat((diagnoses.loc[diagnoses["icd_code"].str.match("58[4-6]\d"), :],
                          diagnoses.loc[diagnoses["icd_code"].str.match("N1[7-9]\d"), :]),0)
# Diabetes
comd["Diabetes"] = pd.concat((diagnoses.loc[diagnoses["icd_code"].str.match("250\d"), :],
                          diagnoses.loc[diagnoses["icd_code"].str.match("E1[0-4]\d"), :]),0)

In [ ]:
for i, var in comd.items():
    
    var_vpn = pd.merge(patients_vpn,var)
    var_vpn[i] = 1
    var_vpn = var_vpn[['stay_id',i]]
    print(var_vpn)
    var_vpn = var_vpn.groupby(["stay_id"]).agg('mean').reset_index()
 
    var_bpn = pd.merge(patients_bpn,var)
    var_bpn[i] = 1
    var_bpn = var_bpn[['stay_id',i]]
    var_bpn= var_bpn.groupby(["stay_id"]).agg('mean').reset_index()

    var_pn = pd.concat((var_vpn, var_bpn),0)
    patients_pn = pd.concat((patients_vpn, patients_bpn),0)
    # print(var_vpn.shape, var_bpn.shape)
    dic[i] = [
        "%s(%.1f)" % (len(set(var_pn["stay_id"])),
                        len(set(var_pn["stay_id"]))/len(set(patients_pn["stay_id"]))*100),
        "%s(%.1f)" % (len(set(var_vpn["stay_id"])),
                        len(set(var_vpn["stay_id"]))/len(set(patients_vpn["stay_id"]))*100),
        "%s(%.1f)" % (len(set(var_bpn["stay_id"])),
                        len(set(var_bpn["stay_id"]))/len(set(patients_bpn["stay_id"]))*100),
        "%.2f" % (stats.chi2_contingency([
                [len(set(var_vpn["stay_id"])), len(set(patients_vpn["stay_id"]))-len(set(var_vpn["stay_id"]))],
                [len(set(var_bpn["stay_id"])), len(set(patients_bpn["stay_id"]))-len(set(var_bpn["stay_id"]))]],correction=False)[1])]
    print(dic[i])

## SOFA 评分

In [35]:
dic = {}
varb = pd.read_csv("pn_bact/patients_sofa.csv", index_col=0)
varv = pd.read_csv("pn_virus/patients_sofa.csv", index_col=0)

for i in ['respiration_24hours', 
          'coagulation_24hours', 
          'liver_24hours',
          'cardiovascular_24hours', 
          'cns_24hours', 
          'renal_24hours',
          'sofa_24hours']:
    
    var_vpn = pd.merge(patients_vpn,varv.loc[varv["hr"]==0])
    var_vpn = var_vpn.loc[var_vpn[i].dropna().index]
    var_bpn = pd.merge(patients_bpn,varb.loc[varb["hr"]==0])
    var_bpn = var_bpn.loc[var_bpn[i].dropna().index]
    print(var_vpn.shape, var_bpn.shape)
    var_pn = pd.concat((var_vpn, var_bpn),0)

    dic[i] = [
          "%.1f(%.1f,%.1f)" % (
                np.mean(var_pn[i]), 
                np.mean(var_pn[i])-np.std(var_pn[i]),
                np.mean(var_pn[i])+np.std(var_pn[i])),
          "%.1f(%.1f,%.1f)" % (
                np.mean(var_vpn[i]), 
                np.mean(var_vpn[i])-np.std(var_vpn[i]),
                np.mean(var_vpn[i])+np.std(var_vpn[i])),
        "%.1f(%.1f,%.1f)" % (
                np.mean(var_bpn[i]), 
                np.mean(var_bpn[i])-np.std(var_bpn[i]),
                np.mean(var_bpn[i])+np.std(var_bpn[i])),
          "%.2f" % (stats.mannwhitneyu(var_vpn[i].T,var_bpn[i].T,alternative='two-sided').pvalue)]
    print(dic[i])
    
# full_vpn = pd.merge(full_vpn, varv.loc[varv["hr"]==0])
# full_bpn = pd.merge(full_bpn, varb.loc[varb["hr"]==0])


(463, 35) (2039, 35)
['1.2(-0.1,2.5)', '0.8(-0.4,2.1)', '1.3(0.0,2.6)', '0.00']
(463, 35) (2039, 35)
['0.5(-0.4,1.3)', '0.6(-0.3,1.5)', '0.4(-0.4,1.2)', '0.00']
(463, 35) (2039, 35)
['0.2(-0.4,0.9)', '0.2(-0.4,0.7)', '0.2(-0.4,0.9)', '0.02']
(463, 35) (2039, 35)
['0.8(-0.5,2.0)', '0.7(-0.5,1.8)', '0.8(-0.5,2.0)', '0.01']
(463, 35) (2039, 35)
['0.3(-0.5,1.0)', '0.2(-0.4,0.9)', '0.3(-0.5,1.1)', '0.43']
(463, 35) (2039, 35)
['0.7(-0.4,1.8)', '0.7(-0.4,1.8)', '0.7(-0.4,1.8)', '0.92']
(463, 35) (2039, 35)
['3.7(0.8,6.5)', '3.2(0.5,5.9)', '3.8(0.9,6.6)', '0.00']


/tmp/ipykernel_2435516/4077084737.py:18: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  var_pn = pd.concat((var_vpn, var_bpn),0)
/tmp/ipykernel_2435516/4077084737.py:18: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  var_pn = pd.concat((var_vpn, var_bpn),0)
/tmp/ipykernel_2435516/4077084737.py:18: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  var_pn = pd.concat((var_vpn, var_bpn),0)
/tmp/ipykernel_2435516/4077084737.py:18: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  var_pn = pd.concat((var_vpn, var_bpn),0)
/tmp/ipykernel_2435516/4077084737.py:18: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  var_pn = pd

In [ ]:
# 30天死亡
var0 = pd.read_csv("pn_virus/patients_death.csv", index_col=0)
var1 = pd.read_csv("pn_bact/patients_death.csv", index_col=0)
var = pd.concat((var0,var1), 0)
i = "30day_death"
var_vpn = pd.merge(patients_vpn,var)
var_vpn = var_vpn.loc[var_vpn[i].dropna().index]
var_bpn = pd.merge(patients_bpn,var)
var_bpn = var_bpn.loc[var_bpn[i].dropna().index]
print(var_vpn.shape, var_bpn.shape)
patients_pn = pd.concat((patients_vpn,patients_bpn),axis = 0)
var_pn = pd.concat((var_vpn, var_bpn),0)

dic[i] = [
        "%s(%.1f)" % (np.sum(var_pn[i]), 
                        np.sum(var_pn[i])/patients_pn.shape[0]*100),
        "%s(%.1f)" % (np.sum(var_vpn[i]), 
                        np.sum(var_vpn[i])/patients_vpn.shape[0]*100),
        "%s(%.1f)" % (np.sum(var_bpn[i]), 
                        np.sum(var_bpn[i])/patients_bpn.shape[0]*100),
        "%.2f" % (stats.chi2_contingency([[np.sum(var_vpn[i]), patients_vpn.shape[0]-np.sum(var_vpn[i])],
                [np.sum(var_bpn[i]), patients_bpn.shape[0]-np.sum(var_bpn[i])]],correction=False)[1])]
pd.DataFrame(dic).T


In [ ]:
# ICU stay
var = pd.read_csv("mimic-iv-2/icu/icustays.csv", index_col=0)
i = "los"
var_vpn = pd.merge(patients_vpn,var)
var_vpn = var_vpn.loc[var_vpn[i].dropna().index]
var_bpn = pd.merge(patients_bpn,var)
var_bpn = var_bpn.loc[var_bpn[i].dropna().index]
print(var_vpn.shape, var_bpn.shape)
var_pn = pd.concat((var_vpn, var_bpn),0)

dic[i] = ["%.1f(%.1f,%.1f)" % (
                np.median(var_pn[i]), 
                np.percentile(var_pn[i], (25), interpolation='midpoint'),
                np.percentile(var_pn[i], (75), interpolation='midpoint')),
          "%.1f(%.1f,%.1f)" % (
                np.median(var_vpn[i]), 
                np.percentile(var_vpn[i], (25), interpolation='midpoint'),
                np.percentile(var_vpn[i], (75), interpolation='midpoint')),
          "%.1f(%.1f,%.1f)" % (
                np.median(var_bpn[i]), 
                np.percentile(var_bpn[i], (25), interpolation='midpoint'),
                np.percentile(var_bpn[i], (75), interpolation='midpoint')),
          "%.2f" % (stats.mannwhitneyu(var_vpn[i].T,var_bpn[i].T,alternative='two-sided').pvalue)]
full_vpn = pd.merge(full_vpn, var_vpn,"outer")
full_bpn = pd.merge(full_bpn, var_bpn,"outer")

# print all

In [130]:
pvdf = pd.DataFrame(dic, index=["总体","病毒组","细菌组","p-value"]).T

,总体,病毒组,细菌组,p-value
age,"67.5(55.5,78.8)","67.6(55.8,79.7)","67.5(55.4,78.6)",0.63
gender,1075(43.0),226(48.8),849(41.6),0.01
weight,"75.9(63.6,92.3)","77.4(65.2,93.8)","75.4(63.1,92.1)",0.04
ventilation_status,1535(61.4),184(39.7),1393(68.3),0.00
Congestive heart failure,838(34.6),182(39.3),698(34.2),0.04
Cardiacarrhythmias,640(26.4),102(22.0),558(27.4),0.02
Pulmonarycirculation,304(12.5),62(13.4),249(12.2),0.54
Hypertention,1483(61.2),284(61.3),1247(61.2),0.98
Chronicpulmonary,901(37.2),185(40.0),747(36.6),0.20
Renal failure,1261(52.0),235(50.8),1072(52.6),0.51


## 实验室检查

In [ ]:
dic = {}
var = pd.read_csv("sofa_data/first_day_lab.csv")
var_vpn = pd.merge(patients_vpn,var)

for i in [
       'hemoglobin_min', 'hemoglobin_max', 
       'platelets_min',
       'wbc_max', 
       'abs_basophils_max','abs_eosinophils_max', 
       'abs_lymphocytes_max', 'abs_monocytes_max',
       'abs_neutrophils_max', 'imm_granulocytes_max', 
       
       'glucose_min','glucose_max', 
       'aniongap_max', 
       'bicarbonate_min', 'bicarbonate_max',
       'calcium_min', 'calcium_max', 
       'chloride_min', 'chloride_max',
       'sodium_min', 'sodium_max', 
       'potassium_min','potassium_max', 
       
       'bun_min',
       'bun_max',
       'd_dimer_max', 
       'fibrinogen_max', 
       'inr_max',
       'pt_max', 
       'ptt_max', 
       
       'albumin_min', 
       'alt_max',
       'alp_max', 
       'ast_max',
       'amylase_max', 
       'creatinine_max', 
       'bilirubin_total_max',
       'bilirubin_direct_max',
       'bilirubin_indirect_max',
       'ck_cpk_max', 
       'ck_mb_max', 
       'ggt_max',
       'ld_ldh_max']:
    var_vpn = pd.merge(patients_vpn,var)
    var_vpn = var_vpn.loc[var_vpn[i].dropna().index]
    var_bpn = pd.merge(patients_bpn,var)
    var_bpn = var_bpn.loc[var_bpn[i].dropna().index]
    print(i ,var_vpn.shape, var_bpn.shape)
    var_pn = pd.concat((var_vpn, var_bpn),0)

    dic[i] = ["%.1f(%.1f,%.1f)" % (
                np.median(var_pn[i]), 
                np.percentile(var_pn[i], (25), interpolation='midpoint'),
                np.percentile(var_pn[i], (75), interpolation='midpoint')),
          "%.1f(%.1f,%.1f)" % (
                np.median(var_vpn[i]), 
                np.percentile(var_vpn[i], (25), interpolation='midpoint'),
                np.percentile(var_vpn[i], (75), interpolation='midpoint')),
          "%.1f(%.1f,%.1f)" % (
                np.median(var_bpn[i]),
                np.percentile(var_bpn[i], (25), interpolation='midpoint'),
                np.percentile(var_bpn[i], (75), interpolation='midpoint')),
          "%.2f" % (stats.mannwhitneyu(var_vpn[i].T,var_bpn[i].T,alternative='two-sided').pvalue)]

In [263]:
full_vpn = pd.merge(full_vpn, var[['subject_id', 'stay_id', 
       'hemoglobin_min', 'hemoglobin_max', 
       'platelets_min',
       'wbc_max', 
       'abs_basophils_max','abs_eosinophils_max', 
       'abs_lymphocytes_max', 'abs_monocytes_max',
       'abs_neutrophils_max', 'imm_granulocytes_max', 
       
       'glucose_min','glucose_max', 
       'aniongap_max', 
       'bicarbonate_min', 'bicarbonate_max',
       'calcium_min', 'calcium_max', 
       'chloride_min', 'chloride_max',
       'sodium_min', 'sodium_max', 
       'potassium_min','potassium_max', 
       
       'bun_min',
       'bun_max',
       'd_dimer_max', 
       'fibrinogen_max', 
       'inr_max',
       'pt_max', 
       'ptt_max', 
       
       'albumin_min', 
       'alt_max',
       'alp_max', 
       'ast_max',
       'amylase_max', 
       'bilirubin_total_max',
       'bilirubin_direct_max',
       'bilirubin_indirect_max',
       'ck_cpk_max', 
       'ck_mb_max', 
       'ggt_max',
       'ld_ldh_max']])
full_bpn = pd.merge(full_bpn, var[['subject_id', 'stay_id', 
       'hemoglobin_min', 'hemoglobin_max', 
       'platelets_min',
       'wbc_max', 
       'abs_basophils_max','abs_eosinophils_max', 
       'abs_lymphocytes_max', 'abs_monocytes_max',
       'abs_neutrophils_max', 'imm_granulocytes_max', 
       
       'glucose_min','glucose_max', 
       'aniongap_max', 
       'bicarbonate_min', 'bicarbonate_max',
       'calcium_min', 'calcium_max', 
       'chloride_min', 'chloride_max',
       'sodium_min', 'sodium_max', 
       'potassium_min','potassium_max', 
       
       'bun_min',
       'bun_max',
       'd_dimer_max', 
       'fibrinogen_max', 
       'inr_max',
       'pt_max', 
       'ptt_max', 
       
       'albumin_min', 
       'alt_max',
       'alp_max', 
       'ast_max',
       'amylase_max', 
       'bilirubin_total_max',
       'bilirubin_direct_max',
       'bilirubin_indirect_max',
       'ck_cpk_max', 
       'ck_mb_max', 
       'ggt_max',
       'ld_ldh_max']])

In [ ]:
a = pd.DataFrame(dic, index=["总体","病毒组","细菌组","p-value"]).T
a.loc[a["p-value"]==a.loc["hemoglobin_min","p-value"],"p-value"] = "＜0.01"
a.index = [i.replace("_", " ") for i in a.index]
a

# 炎性指标

In [ ]:
var = pd.read_csv("sofa_data/inflammation.csv", index_col=0)
i = "crp"
var_vpn = pd.merge(patients_vpn,var)
var_vpn = var_vpn.loc[var_vpn[i].dropna().index]
var_bpn = pd.merge(patients_bpn,var)
var_bpn = var_bpn.loc[var_bpn[i].dropna().index]
print(var_vpn.shape, var_bpn.shape)
var_pn = pd.concat((var_vpn, var_bpn),0)

crp = ["%.1f(%.1f,%.1f)" % (
                np.median(var_pn[i]), 
                np.percentile(var_pn[i], (25), interpolation='midpoint'),
                np.percentile(var_pn[i], (75), interpolation='midpoint')),
          "%.1f(%.1f,%.1f)" % (
                np.median(var_vpn[i]), 
                np.percentile(var_vpn[i], (25), interpolation='midpoint'),
                np.percentile(var_vpn[i], (75), interpolation='midpoint')),
          "%.1f(%.1f,%.1f)" % (
                np.median(var_bpn[i]), 
                np.percentile(var_bpn[i], (25), interpolation='midpoint'),
                np.percentile(var_bpn[i], (75), interpolation='midpoint')),
          "%.2f" % (stats.mannwhitneyu(var_vpn[i].T,var_bpn[i].T,alternative='two-sided').pvalue)]
var_vpn = pd.merge(var_vpn, var_vpn[["stay_id","charttime"]].groupby(['stay_id']).agg('min'))
full_vpn = pd.merge(full_vpn, var_vpn[["stay_id","crp"]], "outer")
var_bpn = pd.merge(var_bpn, var_bpn[["stay_id","charttime"]].groupby(['stay_id']).agg('min'))
full_bpn = pd.merge(full_bpn, var_bpn[["stay_id","crp"]], "outer")

# 筛选并保存大表

In [284]:
full_vpn.to_csv("pn_virus/full_vpn.csv",header=True, index=True)
full_bpn.to_csv("pn_bact/full_bpn.csv",header=True, index=True)